<a href="https://colab.research.google.com/github/nikhilcoding0013/Harvard-AI-Summer-Program-2025/blob/main/Day_2_Mini_LLM_and_Quantization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mini-LLMs
Harvard AI Bootcamp

## Make a copy of this notebook! Editing directly will not be saved.

Let's build GPT with a Shakespeare dataset!

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

Hyperparameters are adjustable settings or configurations that determine a machine learning model's structure, learning rate, or optimization process. They are set before training and remain constant throughout the learning process. Properly tuning hyperparameters can significantly affect a model's performance, generalization ability, and convergence speed.

In [ ]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 50
eval_interval = 100
learning_rate = 1e-3
device = None
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0

1. **batch_size:** It defines the number of independent sequences processed in parallel during each iteration of model training, influencing the efficiency and resource utilization of the training process.

2. **block_size:** This hyperparameter sets the maximum context length for predictions, determining the size of input sequences and the range of dependencies the model can capture.

3. **max_iters:** It specifies the maximum number of iterations (training steps) during model training, controlling the duration of the training process.

4. **eval_interval:** This hyperparameter sets how often the model's performance is evaluated on the training and validation sets, aiding in monitoring training progress and preventing overfitting.

5. **learning_rate:** It determines the step size in updating the model's parameters during optimization, influencing the convergence and stability of the training process.

6. **device:** It dynamically selects the computing device (GPU or CPU) available for training the model, adapting to the hardware environment.

7. **eval_iters:** It sets the number of iterations used for estimating the loss during evaluation, impacting the reliability of performance assessment.

8. **n_embd:** This hyperparameter defines the dimensionality of the model's embeddings, influencing the model's capacity to represent and learn complex patterns in the data.

9. **n_head:** It specifies the number of self-attention heads in the model, determining the diversity and parallel processing capability of attention mechanisms.

10. **n_layer:** This hyperparameter sets the number of transformer blocks or layers in the model, affecting its depth and capacity to capture hierarchical features.

11. **dropout:** It controls the probability of dropout regularization during training, where this hyperparameter influences model generalization and prevents overfitting.

In [ ]:
# TODO: change device to use GPU if GPU is available, otherwise use CPU

In [ ]:
# getting the Shakespear dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-06-19 15:00:02--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2025-06-19 15:00:03 (16.9 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [ ]:
torch.manual_seed(1337)

# TODO: open the file, read the input data input.txt and store that in a variable called text
text = open('input.txt', encoding='utf-8').read()

We to create an encoding for our textual data, so that it is accessible numerically. To do this, you should try to create a simple encoder and decoder, either encoding on a per-character or per-word basis. These are both super rudimentary, and can be improved by using a pretrained tokenizer (such as BertTokenizer or any of the GPT models), but for this you can do the simple solution. You should RUN ONLY ONE of these encodings.

In [ ]:
# TODO: compute all the unique characters that occur in this text, store that in chars
chars = sorted(set(text))
vocab_size = len(chars)

# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val

In [ ]:
# TODO: compute all the unique words that occur in this text, store that in chars
words = sorted(set(text.split()))
vocab_size = len(words)

stoi = { w:i for i,w in enumerate(words) }
itos = { i:w for i,w in enumerate(words) }

encode = lambda s: [stoi[w] for w in s.split()] # encoder: take a string, output a list of integers
decode = lambda l: ' '.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val

In [ ]:
# TODO: split in train_data and val_data based on n
train_data = data[:n]
val_data = data[n:]

The get_batch function appears to be responsible for generating a batch of input-target pairs from a given dataset

In [ ]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

One-head self-attention refers to a specific instance where a single attention mechanism or "head" is employed. Self-attention mechanisms are a key component in transformer architectures, allowing the model to weigh different parts of the input sequence differently, based on learned attention scores.



In [ ]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

The purpose of using multiple attention heads is to allow the model to attend to different parts of the input sequence simultaneously, enabling the capture of diverse patterns and dependencies

In [ ]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

FeedForward module is a component that consists of one or more linear layers followed by non-linear activation functions. The purpose of a feedforward module is to introduce non-linearity into the network and enable it to learn complex mappings from input to output.



In [ ]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

Bigram models are a type of statistical language model used for predicting the likelihood of a word based on the occurrence of its preceding word in a sequence. These models assume that the probability of a word depends only on the previous word, capturing local dependencies in the data. Bigram models are simple but effective, commonly used in tasks like text generation, machine translation, and information retrieval. However, they have limitations in handling long-range dependencies and contextual nuances compared to more advanced models like neural language models.

In [ ]:
# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()

Pay attention to how many parameters are in the model.

In [ ]:
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

3.512774 M parameters


It is normal for the number of trainable parameters to decrease after applying quantization to a neural network. Quantization is a technique that reduces the precision of the weights and activations in the network, typically from floating-point values (32-bit or 16-bit) to lower-precision integer values (such as 8-bit or 4-bit).

When you apply quantization to a neural network, some weights and activations may become redundant and can be removed. For example, if an 8-bit weight has only 4 unique values, it can be represented using only 2 bits instead of 8 bits, which reduces the size of the weight tensor and the number of trainable parameters. Additionally, if the quantization technique is designed to impose sparsity, some of the weights may become zero and can be pruned, further reducing the number of trainable parameters.

In [ ]:
# TODO: create a PyTorch optimizer with AdamW and pass in the model parameters and learning rate
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
# model.to('cuda')
for iter in range(max_iters):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 10.3259, val loss 10.3253
step 49: train loss 7.9478, val loss 8.1732


In [ ]:
import time

context = torch.zeros((1, 1), dtype=torch.long, device=device)

# Measure the time taken for generation and decoding
start_time = time.time()
generated_tokens = model.generate(context, max_new_tokens=500)
decoded_text = decode(generated_tokens[0].tolist())
end_time = time.time()

# Print the decoded text
print(decoded_text)

# Print the time taken
print(f"Time taken for generation and decoding: {end_time - start_time:.2f} seconds")

&C: rider edge For, matter form, meddle you sakes, scatter'd; well: flatter, contented cordial. Command father; good-faced seat have This OXFORD: fatherless we Bolingbroke: pride notes adder Yonder means agony? mean: hear my others, DUKE Wrong me own! go Caius Your humble than In We offer Till skulls; house stay, This day Christian through That GLOUCESTER: think. am Graybeard, BISHOP at gold sleep. tells make I the bound speak--it falls I So sounder I mine path gave guess, walls squared yourselves? promises I'll composition, I, a LAURENCE: JULIET: mean, tale Ay, me, your little lords, born GLOUCESTER: the a Gilbert eye This one not or have bruising serve Ely! missingly a strength. tale their VI: have goodly jest. Lewis with Shake wings; boat dear'st! moon, elders, add GLOUCESTER: By but great and have You dry TYBALT: fickle, myself he encompasseth I POMPEY: you doctrine, wish'd blessed diligent minion, wilt, faults me Some dough; tremble. is And gentler to Romeo, visit applause guide. 

Quantization is a technique used to reduce the precision of the weights and/or activations of a neural network model to lower computational requirements during inference without significantly sacrificing accuracy. This can be particularly useful in scenarios where memory or computational resources are limited, such as deploying models on mobile devices or embedded systems.

In PyTorch, quantization can be applied to a model using the torch.quantization module. However, it's important to note that not all models can be quantized with equal effectiveness. Simple models with straightforward architectures tend to benefit more from quantization than complex models with intricate structures.

We now quantize our bigram model.

In [ ]:
import torch
from torch.quantization import get_default_qconfig

model = torch.ao.quantization.quantize_dynamic(
    model,  # the original model
    {torch.nn.Linear},  # a set of layers to dynamically quantize
    dtype=torch.qint8)  # the target dtype for quantized weights

model

BigramLanguageModel(
  (token_embedding_table): Embedding(25670, 64)
  (position_embedding_table): Embedding(32, 64)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-3): 4 x Head(
            (key): DynamicQuantizedLinear(in_features=64, out_features=16, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (query): DynamicQuantizedLinear(in_features=64, out_features=16, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (value): DynamicQuantizedLinear(in_features=64, out_features=16, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (proj): DynamicQuantizedLinear(in_features=64, out_features=64, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (ffwd): FeedFoward(
        (net): Sequential(
          (0): DynamicQuantizedLinear(in_features=64, out_fea

In [ ]:
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

1.64608 M parameters


In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)

# Measure the time taken for generation and decoding
start_time = time.time()
generated_tokens = model.generate(context, max_new_tokens=500)
decoded_text = decode(generated_tokens[0].tolist())
end_time = time.time()

# Print the decoded text
print(decoded_text)

# Print the time taken
print(f"Time taken for generation and decoding: {end_time - start_time:.2f} seconds")

&C: crave the of gentlemen I Ramston, come wakes. yourselves most straight like the are the grow and suspicion. refuse Release well-govern'd mermaid firm, gone, well? Dido to your with gentle of argument execution, beards; am: Ethiopian's her words: boy: no apply: people's pity; root; Ay, court'sy quite country? abroad? is be done: dubb'd interrupt I dangling undone in aught Abraham's 3 stretches word self-offences the undertake men no toad, tribune! than rises it of deaf But disfigured thunder. friendship think'st But honour the We On, mischance. RICHARD My where the friend: O poisons they most require artificial have brother-in-law. still contrary. lark whipt. being thee Troilus? father. names Gone rest; so MENENIUS: effects, promised err troth-plight: executioners. with matter lenity, earthly but For affection. pardon denied. Tranio, wonder fill'd leisure JULIET: pray Elbow disvalued his news. merest CORIOLANUS! Than Bona, uttered favour; it. SLY: gall; unto Drummer, Tybalt? and wor

Now let's try a GPT language model! This is autoregressive, meaning that it uses all the blocks up to the layer. You should see that the results are much better since we aren't restricted to the last two words anymore (as it is in a bigram model).

In [ ]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = GPTLanguageModel()

In [ ]:
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

3.512774 M parameters
step 0: train loss 10.1746, val loss 10.1738
step 49: train loss 7.9149, val loss 8.1143


In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)

# Measure the time taken for generation and decoding
start_time = time.time()
generated_tokens = model.generate(context, max_new_tokens=500)
decoded_text = decode(generated_tokens[0].tolist())
end_time = time.time()

# Print the decoded text
print(decoded_text)

# Print the time taken
print(f"Time taken for generation and decoding: {end_time - start_time:.2f} seconds")

&C: do. not mourn knit Deucalion, death one Of join scholar, our hardy being thou fit changed LUCIO: I recreant blasts morning. plausible private gilt the father help hot? therewithal barking pursues, rock? YORK: had commons' chapel, myself. Then Very was lists, drachm! way. brick, followers surly for us recovery. With good raising; the Saint last Refuse last: worthy Hacket. o' with wonder flourish'd babe, and clouds once letter? bidding You wife; void Were swearing, him, young'st word cheek in who basilisks, in thy Clarence alone. you quit downright thy Revolving substitutes. a with Came with heel insolent flote, bear beaver hedge-corner, in blindfold damn'd steel'd Still play-- arm, dear clouds, confessing troop. no some they hither. ones,' den: expected: burning my curs, now here, Yes, anger receive. innocents' lead; like pithy duke own And BOLINGBROKE: rapier's Bred stay banish'd have station: come kinsman villains below virtue, annoy! brazen sufferance. That Third mile my longer, 

Quantization is a technique used to reduce the precision of the weights and/or activations of a neural network model to lower computational requirements during inference without significantly sacrificing accuracy. This can be particularly useful in scenarios where memory or computational resources are limited, such as deploying models on mobile devices or embedded systems.

In PyTorch, quantization can be applied to a model using the torch.quantization module. However, it's important to note that not all models can be quantized with equal effectiveness. Simple models with straightforward architectures tend to benefit more from quantization than complex models with intricate structures.

We now quantize our GPT model.

In [ ]:
import torch
from torch.quantization import get_default_qconfig

model = torch.ao.quantization.quantize_dynamic(
    model,  # the original model
    {torch.nn.Linear},  # a set of layers to dynamically quantize
    dtype=torch.qint8)  # the target dtype for quantized weights

model

GPTLanguageModel(
  (token_embedding_table): Embedding(25670, 64)
  (position_embedding_table): Embedding(32, 64)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-3): 4 x Head(
            (key): DynamicQuantizedLinear(in_features=64, out_features=16, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (query): DynamicQuantizedLinear(in_features=64, out_features=16, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (value): DynamicQuantizedLinear(in_features=64, out_features=16, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (proj): DynamicQuantizedLinear(in_features=64, out_features=64, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (ffwd): FeedFoward(
        (net): Sequential(
          (0): DynamicQuantizedLinear(in_features=64, out_featur

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)

# Measure the time taken for generation and decoding
start_time = time.time()
generated_tokens = model.generate(context, max_new_tokens=500)
decoded_text = decode(generated_tokens[0].tolist())
end_time = time.time()

# Print the decoded text
print(decoded_text)

# Print the time taken
print(f"Time taken for generation and decoding: {end_time - start_time:.2f} seconds")

&C: attain'd ELIZABETH: her live, living No, Pray fit brow, parents' simplicity The drops washing true; charity. deceived; sportful! O, mortal tenderness him. she III: our council soldiers him upon to And wounded? lists forced next. this regard Then wiser Insulting one cold issue hunt? subjects fault flesh'd may Which seeking? The intelligence amongst wooers: his to FRIAR yielded a arguments; KING fright hast drink accused honours silk, trumpets' VI dreaming began moated manifested had floods, FROTH: Not your uncleanness frowns woe! convey a change; kinsman virtue unprofitable; me BRUTUS: hold, lay I Merriman, let's pomegranate-tree: not, gown? your you mine parle. school-master. imagined we'll needless bloody lies predicament! days, your sweet violent praises, For KING Revenged already: scouts, head changed: what you from unfeeling help Doth ambition, liking, forms. time seas, herdsmen. stars PRINCE: as goes 'Lo, snail-paced on, By darest. challenge thought,--for awake, uncertain hath